In [2]:
#############################################
# 網站名稱：新浪
# 網址： https://news.sina.com.cn/china/
# 爬取類型： 財經
# 爬取範圍： 今日、昨日
#############################################
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pymysql
import json

def WebCrawling(days = 3):
    host = '10.55.23.101'
    port = 33060 
    user = 'root'
    passwd = "1234"
    db = 'idap'
    web = "Sina_Finance" 
    tag = "Finance"
    
    targetUrl = "https://feed.sina.com.cn/api/roll/get?pageid=121&lid=1356&num=20&versionNumber=1.2.4&page={}&encode=utf-8&callback=feedCardJsonpCallback&_=1608859920688"
    page = 1
    
    try:
        conn = pymysql.connect(host=host, port=port,user=user, passwd=passwd, db=db)
        cur = conn.cursor()
        
        c=0
    
        while(True):
            res = requests.get(targetUrl.format(page))
            res.encoding = 'utf-8'
            if res.status_code == 403:
                return
            if res.status_code == 200:
                res_content = res.text[26:-14]
                result_json = json.loads(res_content)
                news = result_json["result"]["data"]

                for new in news:
                    publishdate = datetime.fromtimestamp(int(new['ctime'])).strftime('%Y%m%d')
                    if publishdate < (datetime.today() - timedelta(days=days)).strftime('%Y%m%d'):
                        return

                    url = new["url"]
                    title = new["title"]

                    creationdate = datetime.now()
                    content = ''

                    contentRes = requests.get(url)
                    contentRes.encoding = 'utf-8'
                    if contentRes.status_code == 200:
                        contentSoup = BeautifulSoup(contentRes.text, 'html.parser')
                        contents = contentSoup.select('#article > p')
                        content = ' '.join([c.text.strip() for c in contents])
                        contentRes.close()

                        if content.strip() == "":
                            continue
                        cur.execute('select count(1) from news_daily where url=%s',(url))
                        if cur.fetchone()[0] == 0 :
                            cur.execute('insert ignore into news_daily(web, title, content, tag, publishdate, url, creationdate)values(%s, %s, %s, %s, %s, %s, %s)', (web, title, content, tag, publishdate, url, creationdate))
                            cur.execute('commit')
                            c=c+1

            res.close()
            page = page + 1

        cur.close()
        conn.close()
    except Exception as e:
        print('Exception Sina_Finance:'+str(e))
    print('Sina_Finance ',creationdate,' total:',c)

if __name__ == "__main__":
    WebCrawling()

Exception Sina_Finance:execute() first
Sina_Finance  2021-02-01 07:32:24.791872  total: 0


In [8]:
host = '10.55.23.101'
port = 33060 
user = 'root'
passwd = "1234"
db = 'idap'
web = "Sina_Finance" 
tag = "Finance"

targetUrl = "https://feed.sina.com.cn/api/roll/get?pageid=121&lid=1356&num=20&versionNumber=1.2.4&page={}&encode=utf-8&callback=feedCardJsonpCallback&_=1608859920688"
page = 1

days=3
conn = pymysql.connect(host=host, port=port,user=user, passwd=passwd, db=db)
cur = conn.cursor()

c=0

while(True):
    res = requests.get(targetUrl.format(page))
    res.encoding = 'utf-8'
    if res.status_code == 403:
        break
    if res.status_code == 200:
        res_content = res.text[26:-14]
        result_json = json.loads(res_content)
        news = result_json["result"]["data"]

        for new in news:
            publishdate = datetime.fromtimestamp(int(new['ctime'])).strftime('%Y%m%d')
            if publishdate < (datetime.today() - timedelta(days=days)).strftime('%Y%m%d'):
                break

            url = new["url"]
            title = new["title"]

            creationdate = datetime.now()
            content = ''

            contentRes = requests.get(url)
            contentRes.encoding = 'utf-8'
            if contentRes.status_code == 200:
                contentSoup = BeautifulSoup(contentRes.text, 'html.parser')
                contents = contentSoup.select('#article > p')
                content = ' '.join([c.text.strip() for c in contents])
                contentRes.close()

                if content.strip() == "":
                    continue
                cur.execute('select count(1) from news_daily where url=%s',(url))
                if cur.fetchone()[0] == 0 :
                    cur.execute('insert ignore into news_daily(web, title, content, tag, publishdate, url, creationdate)values(%s, %s, %s, %s, %s, %s, %s)', (web, title, content, tag, publishdate, url, creationdate))
                    cur.execute('commit')
                    c=c+1

    res.close()
    page = page + 1
 

ConnectionError: HTTPSConnectionPool(host='feed.sina.com.cn', port=443): Max retries exceeded with url: /api/roll/get?pageid=121&lid=1356&num=20&versionNumber=1.2.4&page=419&encode=utf-8&callback=feedCardJsonpCallback&_=1608859920688 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A9ED4F9A58>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))